In [1]:
import folium

In [5]:
from google.colab import files
uploaded = files.upload()


Saving projections.csv to projections.csv


In [127]:
import pandas as pd
projections= pd.read_csv("projections.csv")
projections = projections.set_index("zip_code")
mask = (10 < projections.sum(axis=1)) & (projections.sum(axis=1) < 20)

projections[mask]

,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,...,2045-03-01,2045-04-01,2045-05-01,2045-06-01,2045-07-01,2045-08-01,2045-09-01,2045-10-01,2045-11-01,2045-12-01
zip_code,,,,,,,,,,,,,,,,,,,,,
1106,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3862,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4220,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4548,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4989,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [128]:
data_rough = projections.loc[1106]
data_rough


,1106
2025-01-01,0
2025-02-01,0
2025-03-01,0
2025-04-01,0
2025-05-01,0
...,...
2045-08-01,0
2045-09-01,0
2045-10-01,1
2045-11-01,1


In [129]:
dates_index = pd.to_datetime(data_rough.index)
formatted_dates = dates_index.strftime('%Y-%m-%dT%H:%M:%S').tolist()
print(f"First 5 formatted dates: {formatted_dates[:5]}")

First 5 formatted dates: ['2025-01-01T00:00:00', '2025-02-01T00:00:00', '2025-03-01T00:00:00', '2025-04-01T00:00:00', '2025-05-01T00:00:00']


In [130]:
import random
# places_placeholder = [''] * len(formatted_dates)
lats_placeholder = []
lons_placeholder = []
for i in range(len(formatted_dates)):
  lats_placeholder.append(40.76191+random.uniform(-0.0013,0.0013))
  lons_placeholder.append(-73.93169+random.uniform(-0.0013,0.0013))
  # lats_placeholder = [40.730610+random.uniform(-0.00023,0.00023)] * len(formatted_dates)
  # lons_placeholder = [-73.9352242+random.uniform(-0.00023,0.00023)] * len(formatted_dates)
icons_placeholder = ["https://img.icons8.com/officel/80/big-ben.png"] * len(formatted_dates)

print(f"Length of formatted_dates: {len(formatted_dates)}")
# print(f"First 5 places_placeholder: {places_placeholder[:5]}")
print(f"First 5 lats_placeholder: {lats_placeholder[:5]}")
print(f"First 5 lons_placeholder: {lons_placeholder[:5]}")
print(f"First 5 icons_placeholder: {icons_placeholder[:5]}")

Length of formatted_dates: 252
First 5 lats_placeholder: [40.76313148996819, 40.76318807147906, 40.76291522315966, 40.76254969550328, 40.76237962557701]
First 5 lons_placeholder: [-73.93199450728746, -73.9326278780553, -73.93113675820712, -73.93256510149487, -73.93148270514682]
First 5 icons_placeholder: ['https://img.icons8.com/officel/80/big-ben.png', 'https://img.icons8.com/officel/80/big-ben.png', 'https://img.icons8.com/officel/80/big-ben.png', 'https://img.icons8.com/officel/80/big-ben.png', 'https://img.icons8.com/officel/80/big-ben.png']


In [131]:
stops = []

for i, date in enumerate(formatted_dates):

    stop_data = {
        'time': date,
        'lat': lats_placeholder[i],
        'lon': lons_placeholder[i],
        'icon': icons_placeholder[i]

    }
    stops.append(stop_data)

print(f"Number of stops created: {len(stops)}")
print("First 3 stops:")
for stop in stops[:3]:
    print(stop)

Number of stops created: 252
First 3 stops:
{'time': '2025-01-01T00:00:00', 'lat': 40.76313148996819, 'lon': -73.93199450728746, 'icon': 'https://img.icons8.com/officel/80/big-ben.png'}
{'time': '2025-02-01T00:00:00', 'lat': 40.76318807147906, 'lon': -73.9326278780553, 'icon': 'https://img.icons8.com/officel/80/big-ben.png'}
{'time': '2025-03-01T00:00:00', 'lat': 40.76291522315966, 'lon': -73.93113675820712, 'icon': 'https://img.icons8.com/officel/80/big-ben.png'}


In [136]:
data_rough = data_rough.reset_index()
data_rough

for i, stop in enumerate(stops):
    stop['evs'] = data_rough.iloc[i][1106]

# print(f"Number of stops created: {len(stops)}")
# print("First 3 stops with 'evs' feature:")
# for stop in stops[20:50]:
#     print(stop)

# for i, stop in enumerate(stops):
#   if stop['evs'] <= 10000000:
#     stops.pop(i)
#     print(len(stops))

# stops

In [137]:
geojson_features = []

for stop in stops:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [stop['lon'], stop['lat']]
        },
        "properties": {
            "time": stop['time'],
            "evs": int(stop['evs'])
        }
    }
    if feature["properties"]["evs"] > 0:
      geojson_features.append(feature)

print(f"Number of GeoJSON features created: {len(geojson_features)}")
print("First 3 GeoJSON features:")
import json
for feature in geojson_features[:3]:
    print(json.dumps(feature, indent=2))




Number of GeoJSON features created: 19
First 3 GeoJSON features:
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93185854221564,
      40.76172815983356
    ]
  },
  "properties": {
    "time": "2036-02-01T00:00:00",
    "evs": 1
  }
}
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93072669208682,
      40.762055838030456
    ]
  },
  "properties": {
    "time": "2036-11-01T00:00:00",
    "evs": 1
  }
}
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93186182638668,
      40.76127984464928
    ]
  },
  "properties": {
    "time": "2037-11-01T00:00:00",
    "evs": 1
  }
}


In [138]:
from folium.plugins import TimestampedGeoJson
print("TimestampedGeoJson imported successfully.")

TimestampedGeoJson imported successfully.


In [142]:
timestamped_geojson_layer = TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': geojson_features
       # 'features': aggregated_geojson_features
    },
    period='P1M',  # One month per step (adjust as needed based on your date frequency)
    auto_play=False,
    loop=False,
    max_speed=1,
    min_speed=0.1,
    date_options='YYYY-MM-DD',
    transition_time=200
)
print("TimestampedGeoJson layer created.")

TimestampedGeoJson layer created.


In [143]:
d = folium.Map(location=[40.76191, -73.93169],zoom_start=16)
timestamped_geojson_layer.add_to(d)
d

In [123]:
aggregated_geojson_features = []
i = 0
while i < len(geojson_features):
    f1 = geojson_features[i]
    if i + 1 < len(geojson_features):
        f2 = geojson_features[i+1]

        combined_lat = (f1['geometry']['coordinates'][1] + f2['geometry']['coordinates'][1]) / 2
        combined_lon = (f1['geometry']['coordinates'][0] + f2['geometry']['coordinates'][0]) / 2
        combined_evs = f1['properties']['evs'] + f2['properties']['evs']
        combined_time = f1['properties']['time'] # Use the time of the first feature for animation

        aggregated_geojson_features.append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [combined_lon, combined_lat]
            },
            "properties": {
                "time": combined_time,
                "evs": combined_evs
            }
        })
        i += 2 # Move to the next pair
    else:
        # If there's an odd number of features, add the last one as is
        aggregated_geojson_features.append(f1)
        i += 1

print(f"Number of aggregated GeoJSON features created: {len(aggregated_geojson_features)}")
print("First 3 aggregated GeoJSON features:")
import json
for feature in aggregated_geojson_features[:3]:
    print(json.dumps(feature, indent=2))

Number of aggregated GeoJSON features created: 10
First 3 aggregated GeoJSON features:
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93185097772331,
      40.7624148805543
    ]
  },
  "properties": {
    "time": "2036-02-01T00:00:00",
    "evs": 2
  }
}
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93251127242891,
      40.76115794695107
    ]
  },
  "properties": {
    "time": "2037-11-01T00:00:00",
    "evs": 2
  }
}
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      -73.93153272623616,
      40.76109107485941
    ]
  },
  "properties": {
    "time": "2039-11-01T00:00:00",
    "evs": 2
  }
}
